In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import transformers
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(64)

# Load the saved models
save_path = 'Training_results/Laura_NegativeOnly/'
max_length = 256
batchsize = 48
saved_model_paths = [os.path.join(save_path, f"final_bert_model_laura_neg.pth") for i in range(5)]
models = []

# Load BERT tokenizer and model
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = transformers.BertModel.from_pretrained('bert-base-uncased')

# Get the target list
target_list = [
    'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage',
    'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible',
    'replacementAssistanceEligible', 'personalPropertyEligible'
]

def haversine(lat1, lon1, lat2, lon2):
    R = 3959
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def get_box_area(lat1, lon1, lat2, lon2):
    side1 = haversine(lat1, lon1, lat1, lon2)
    side2 = haversine(lat1, lon1, lat2, lon1)
    return side1 * side2

# Get the target list if not provided
def get_target_list(target_list=[]):
    if not target_list:
        target_list = [
            'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage',
            'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible',
            'replacementAssistanceEligible', 'personalPropertyEligible'
        ]
    return target_list

def grouped_tweets_to_dict(tweet_grouped):
    return {int(name): group['text'] for name, group in tweet_grouped}

# CustomDataset without labels (for inference only)
class InferenceDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0)
        }

class BERTDeepMultiHeadClassifier(nn.Module):
    def __init__(self, num_targets=11, hidden_dim=256):
        super(BERTDeepMultiHeadClassifier, self).__init__()
        self.bert = model_bert
        self.drop = nn.Dropout(0.3)

        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.bert.config.hidden_size, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, 1)
            ) for _ in range(num_targets)
        ])

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.drop(pooled_output)
        return torch.cat([torch.sigmoid(head(x)) for head in self.heads], dim=1)

map_location = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for path in saved_model_paths:
    model = BERTDeepMultiHeadClassifier(num_targets=len(target_list)).to(map_location)
    model.load_state_dict(torch.load(path, map_location=map_location))
    model.eval()
    models.append(model)

size_threshold = 80

tweets2 = pd.read_csv('organized_with_zipcode.csv')  # read in massive tweets dataset
tweets_beryl = tweets2[(tweets2.storm_name == 'beryl')]
bboxes_useful = tweets_beryl.place_bbox.apply(lambda x: [[float(i.strip('(').strip(')')) for i in x.split(', ')][i] for i in [1,0,3,2]])
bbu_areas = bboxes_useful.apply(lambda x: get_box_area(*x))
tweets_beryl = tweets_beryl.loc[((tweets_beryl.geo.apply(lambda x: 'Point' in str(x))) | (bbu_areas < size_threshold)),:]  # since i'm using iloc i think the indices will match
tweet_grouped_beryl = tweets_beryl.groupby('zip_code')

tweet_dict = {int(name): '\n'.join(group['text'].to_list()) for name, group in tweet_grouped_beryl}

# Prepare data
zip_codes = list(tweet_dict.keys())
texts = [tweet_dict[zc] for zc in zip_codes]

# Create inference dataset and loader
inference_dataset = InferenceDataset(texts=texts, tokenizer=tokenizer, max_len=max_length)
inference_loader = DataLoader(inference_dataset, batch_size=batchsize, shuffle=False)

# Store predictions
all_preds = []

with torch.no_grad():
    for batch in tqdm(inference_loader, desc="Making predictions", unit="batch"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        fold_preds = []
        for model in models:
            outputs = model(input_ids, attention_mask)
            fold_preds.append(outputs.cpu().numpy())

        fold_preds = np.stack(fold_preds, axis=0)  # Shape: (num_models, batch_size, num_targets)
        all_preds.append(fold_preds)

# Combine predictions
all_preds = np.concatenate(all_preds, axis=1)  # Shape: (num_models, total_samples, num_targets)
mean_preds = all_preds.mean(axis=0)            # Shape: (total_samples, num_targets)
binary_preds = (mean_preds > 0.5).astype(int)

target_list = [
    'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage',
    'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible',
    'replacementAssistanceEligible', 'personalPropertyEligible'
]

# Save to files
mean_preds_df = pd.DataFrame(mean_preds, columns=target_list)
mean_preds_df.insert(0, "zip_code", zip_codes)
mean_preds_path = os.path.join(save_path, "laura_bert_on_beryl_mean_predictions.xlsx")
mean_preds_df.to_excel(mean_preds_path, index=False)

binary_preds_df = pd.DataFrame(binary_preds, columns=target_list)
binary_preds_df.insert(0, "zip_code", zip_codes)
binary_preds_path = os.path.join(save_path, "laura_bert_on_beryl_binary_predictions.xlsx")
binary_preds_df.to_excel(binary_preds_path, index=False)

print(f"Mean predictions saved to {mean_preds_path}")
print(f"Binary predictions saved to {binary_preds_path}")

Making predictions: 100%|██████████| 6/6 [01:16<00:00, 12.75s/batch]


Mean predictions saved to Training_results/Laura_NegativeOnly/laura_bert_on_beryl_mean_predictions.xlsx
Binary predictions saved to Training_results/Laura_NegativeOnly/laura_bert_on_beryl_binary_predictions.xlsx


In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import transformers
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(64)

# Load the saved models
save_path = 'Training_results/Laura_NegativeOnly/'
saved_model_paths = [os.path.join(save_path, f"final_bert_model_laura_neg.pth") for i in range(5)]
models = []

# Load BERT tokenizer and model
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = transformers.BertModel.from_pretrained('bert-base-uncased')

# Get the target list
target_list = [
    'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage', 
    'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible', 
    'replacementAssistanceEligible', 'personalPropertyEligible'
]

def haversine(lat1, lon1, lat2, lon2):
    R = 3959  
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def get_box_area(lat1, lon1, lat2, lon2):
    side1 = haversine(lat1, lon1, lat1, lon2)
    side2 = haversine(lat1, lon1, lat2, lon1)
    return side1 * side2

# Get the target list if not provided
def get_target_list(target_list=[]):
    if not target_list:
        target_list = [
            'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage', 
            'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible', 
            'replacementAssistanceEligible', 'personalPropertyEligible'
        ]
    return target_list

# Define dataset class to handle tokenization and data loading
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenizing the text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.float32)
        }

class BERTDeepMultiHeadClassifier(nn.Module):
    def __init__(self, num_targets=11, hidden_dim=256):
        super(BERTDeepMultiHeadClassifier, self).__init__()
        self.bert = model_bert
        self.drop = nn.Dropout(0.3)
        
        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.bert.config.hidden_size, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, 1)
            ) for _ in range(num_targets)
        ])
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.drop(pooled_output)
        return torch.cat([torch.sigmoid(head(x)) for head in self.heads], dim=1)

# Load all trained models
map_location = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for path in saved_model_paths:
    model = BERTDeepMultiHeadClassifier(num_targets=len(target_list)).to(map_location)
    model.load_state_dict(torch.load(path, map_location=map_location))
    model.eval()
    models.append(model)

size_threshold = 80

tweets2 = pd.read_csv('organized_with_zipcode.csv')  # read in massive tweets dataset
tweets_beryl = tweets2[(tweets2.storm_name == 'beryl')]
bboxes_useful = tweets_beryl.place_bbox.apply(lambda x: [[float(i.strip('(').strip(')')) for i in x.split(', ')][i] for i in [1,0,3,2]])
bbu_areas = bboxes_useful.apply(lambda x: get_box_area(*x))
tweets_beryl = tweets_beryl.loc[((tweets_beryl.geo.apply(lambda x: 'Point' in str(x))) | (bbu_areas < size_threshold)),:]  # since i'm using iloc i think the indices will match
tweet_grouped_beryl = tweets_beryl.groupby('zip_code')

targets_beryl = pd.read_csv('disaster_4798.csv')
target_grouped_beryl = targets_beryl.groupby('damagedZipCode')
    
tweet_dict = {int(name): group['text'] for name, group in tweet_grouped_beryl}
target_dict = {int(name): group[target_list] for name, group in target_grouped_beryl}

intersecting_zips = list(set(target_dict.keys()) & set(tweet_dict.keys()))
paired_data = {
    name: [target_dict[name].sum().apply(lambda x: 1 if x > 0 else 0), tweet_dict[name]] for name in intersecting_zips
}

# Prepare dataset for inference
texts = ['\n'.join(v[1].to_list()) for v in paired_data.values()]
labels_ = [v[0] for v in paired_data.values()]
zip_codes = list(paired_data.keys())

test_dataset = CustomDataset(texts, labels_, tokenizer, max_len=512)
test_loader = DataLoader(test_dataset, batch_size=24, shuffle=False)

# Store predictions
all_preds = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Making predictions", unit="batch"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        fold_preds = []
        for model in models:
            outputs = model(input_ids, attention_mask)
            fold_preds.append(outputs.cpu().numpy())

        fold_preds = np.stack(fold_preds, axis=0)  # Shape: (num_models, batch_size, num_targets)
        all_preds.append(fold_preds)

# Convert list to numpy array
all_preds = np.concatenate(all_preds, axis=1)  # Shape: (num_models, total_samples, num_targets)
mean_preds = all_preds.mean(axis=0)  # Mean prediction probabilities (Shape: total_samples, num_targets)
binary_preds = (mean_preds > 0.5).astype(int)  # Convert probabilities to binary predictions

# Convert labels to numpy array
true_labels = np.vstack(labels_)

# Compute evaluation metrics
# metrics_dict = {'Target': target_list, 'Accuracy': [], 'F1_Score': [], 'Precision': [], 'Recall': []}
metrics_dict = {'Target': target_list, 'Accuracy': [], 'F1_Score': [], 'Precision': [], 'Recall': [], 'Support': []}

for i in range(len(target_list)):
    metrics_dict['Accuracy'].append(accuracy_score(true_labels[:, i], binary_preds[:, i]))
    metrics_dict['F1_Score'].append(f1_score(true_labels[:, i], binary_preds[:, i]))
    metrics_dict['Precision'].append(precision_score(true_labels[:, i], binary_preds[:, i]))
    metrics_dict['Recall'].append(recall_score(true_labels[:, i], binary_preds[:, i]))
    metrics_dict['Support'].append(int(true_labels[:, i].sum()))

# Compute and append average metrics
metrics_dict['Target'].append('Average')
metrics_dict['Accuracy'].append(np.mean(metrics_dict['Accuracy']))
metrics_dict['F1_Score'].append(np.mean(metrics_dict['F1_Score']))
metrics_dict['Precision'].append(np.mean(metrics_dict['Precision']))
metrics_dict['Recall'].append(np.mean(metrics_dict['Recall']))
metrics_dict['Support'].append('–')

# Convert to DataFrame and save
metrics_df = pd.DataFrame(metrics_dict)
metrics_df_path = os.path.join(save_path, "bert_metrics_on_beryl.xlsx")
metrics_df.to_excel(metrics_df_path, index=False)

print(f"Metrics saved to {metrics_df_path}")

/var/folders/89/9glrgdc55d79wbzwgyfj0w7r0000gn/T/ipykernel_84212/811228265.py:126: DtypeWarning: Columns (9,13,28,46,49) have mixed types. Specify dtype option on import or set low_memory=False.
  targets_beryl = pd.read_csv('disaster_4798.csv')
Making predictions:   0%|          | 0/4 [00:00<?, ?batch/s]/var/folders/89/9glrgdc55d79wbzwgyfj0w7r0000gn/T/ipykernel_84212/811228265.py:85: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'labels': torch.tensor(label, dtype=torch.float32)
Making predictions: 100%|██████████| 4/4 [00:50<00:00, 12.54s/batch]

Metrics saved to Training_results/Laura_NegativeOnly/bert_metrics_on_beryl.xlsx



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/